In [1]:
from utils import load_results_policies
from games import get_game
import matplotlib.pyplot as plt
import ipywidgets as wid
import numpy as np
import seaborn as sns
from itertools import product
sns.set()


In [2]:
@wid.interact(p=(0, 1, 0.1), a=(1, 10, 1), step_type=['lola','naive'])
def plot_all_res(p=0.5, a=3, step_type='naive'):
    save_folder = "DistInfComms_p{}_a{}_{}_0".format(p, a, step_type)
    game = get_game("DistInfComms", p=p, a=a)
    _, _, p1s, p2s = load_results_policies(save_folder)
    p1s = np.array(p1s)
    p2s = np.array(p2s)
    
    ### Mutual information between signals and types, and signals and actions
    plt.figure(figsize=(12, 12))
    MIts = [game.MI_ts(p1, p2) for p1, p2 in zip(p1s, p2s)]
    MIas = [game.MI_as(p1, p2) for p1, p2 in zip(p1s, p2s)]

    ind = 0
    titles = ["I(t1;s1)", "I(t2;s2)", "I(a1;(s1, s2))", "I(a2;(s1, s2))"]
    for MIs in [MIts, MIas]:
        MI1s, MI2s = [ i for i, j in MIs ], [ j for i, j in MIs]
        for MI in [MI1s, MI2s]:
            ind += 1
            plt.subplot(2, 2, ind)
            plt.xlim(0,.7)
            plt.ylim(0, 40)
            plt.xlabel(titles[ind-1])
            sns.histplot(x=MI, binwidth=0.1)
    plt.tight_layout()

interactive(children=(FloatSlider(value=0.5, description='p', max=1.0), IntSlider(value=3, description='a', ma…

In [14]:
@wid.interact(a=(1, 10, 1), step_type=['lola','naive'], babble=True, coord=True, leader=True, comm=True, other=True)
def plot_all_res(a=3, step_type='naive', babble=True, coord=True, leader=True, comm=True, other=True):
    ps = [i/10 for i in range(0,11)]

    using = [babble, coord, leader, comm, other]
    names = ["Babble", "Coord", "Leader", "Comm", "Other"]
    colours = {names[n]:'C{}'.format(n) for n in range(len(names))}
    names = [names[n] for n in range(len(names)) if using[n]]

    all_xs = []
    all_ys = []
    hue = []
    for p in ps:
        save_folder = "DistInfComms_p{}_a{}_{}_0".format(p, a, step_type)
        game = get_game("DistInfComms", p=p, a=a)
        _, _, p1s, p2s = load_results_policies(save_folder)
        p1s = np.array(p1s)
        p2s = np.array(p2s)

        res = game.get_equilibria(p1s, p2s)
        res = [res[r] for r in range(len(res)) if using[r]]
        all_xs += [p]*len(names)
        all_ys += [len(r) for r in res]
        hue += names

    plt.figure(figsize=(15, 4))
    plt.ylim(0,25)
    plt.xlim(0,1)
    plt.ylabel("Count")
    plt.xlabel("p")
    sns.barplot(x=all_xs, y=all_ys, hue=hue, palette=colours)


interactive(children=(IntSlider(value=3, description='a', max=10, min=1), Dropdown(description='step_type', in…